In [305]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

In [306]:
# Load the Excel file
file_name = "SES_2024_tidy.xlsx.coredownload.xlsx"
sheet_name = "T3.5"

# Read the data from the Excel file
df = pd.read_excel(file_name, sheet_name=sheet_name)
df = df.fillna(0)

# Treat the random 's' as 0
df['Kwh_Per_Acc_Cleaned'] = df['Kwh Per Acc'].apply(lambda x: 0 if isinstance(x, str) and 's' in x else x) 

# Filter the data for the year 2024 and 'Dwelling Type' as 'overall'
filtered_data = df[(df['Year'] == 2024) & (df['Dwelling Type'] == 'Overall') & (df['Month'] == 'Annual')]

# Calculate total consumption per region
regions = ['Central Region', 'East Region', 'North East Region', 'North Region', 'West Region']
demand_kW_per_region = {
    region: filtered_data[filtered_data['Region'] == region]['Kwh_Per_Acc_Cleaned'].astype(float).sum()
    for region in regions
}

demand_kW_per_region

{'Central Region': 12698.599999999999,
 'East Region': 3884.1000000000004,
 'North East Region': 3428.7000000000003,
 'North Region': 3354.5,
 'West Region': 2853.1000000000004}

In [307]:
df_carpark = pd.read_csv('Checkmark2.csv')

# Calculate the number of car parks per region
carpark_per_region = {
    region: df_carpark[df_carpark['region'] == region].shape[0]
    for region in regions
}

# Output the result
carpark_per_region

{'Central Region': 541,
 'East Region': 400,
 'North East Region': 493,
 'North Region': 303,
 'West Region': 506}

In [308]:
# Define other parameters
types_of_panels = ['Monocrystalline', 'Polycrystalline', 'Thin-film']
efficiency = {'Monocrystalline': 0.15, 'Polycrystalline': 0.13, 'Thin-film': 0.07}
var_cost = {'Monocrystalline': 266.125, 'Polycrystalline': 265.85, 'Thin-film': 265.5}
fixed_cost = 39457.75
available_space_m2 = 2559.5

'''Sensitivity analysis'''

# available_space_m2 = 2559.5 * 1.1 # increase by 10%
# available_space_m2 = 2559.5 * 0.9 # decrease by 10%
# available_space_m2 = 1808 # lower bound
# available_space_m2 = 3311 # upper bound

budget_limit = 40_000_000 # Budget limit in dollars

# Calculate the total area by region constraint
area_by_region_constraint = {
    region: carpark_per_region[region] * available_space_m2
    for region in carpark_per_region
}

area_by_region_constraint

{'Central Region': 1384689.5,
 'East Region': 1023800.0,
 'North East Region': 1261833.5,
 'North Region': 775528.5,
 'West Region': 1295107.0}

In [309]:
# Create a new model
model = gp.Model("solar_optimization")

# Decision variables
x = model.addVars(regions, types_of_panels, vtype=GRB.INTEGER, name="x")
y = model.addVars(regions, vtype=GRB.BINARY, name="y")

In [310]:
model.setObjective(
    gp.quicksum(fixed_cost * y[region] + gp.quicksum(var_cost[panel] * x[region, panel] for panel in types_of_panels) for region in regions),
    GRB.MINIMIZE
)

In [311]:
# Add constraints
for region in regions:
    model.addConstr(gp.quicksum(efficiency[panel] * x[region, panel] for panel in types_of_panels) >= demand_kW_per_region[region], f"Demand_Constraint_{region}")
    model.addConstr(gp.quicksum(15*x[region, panel] for panel in types_of_panels) <= area_by_region_constraint[region], f"Space_Constraint_{region}")
    model.addConstr(fixed_cost * y[region] + gp.quicksum(var_cost[panel] * x[region, panel] for panel in types_of_panels) <= budget_limit, f"Budget_Constraint_{region}")
    for panel in types_of_panels:
        model.addConstr(x[region, panel] <= GRB.INFINITY * y[region], f"Big_M_Constraint_{region}_{panel}")
        model.addConstr(x[region, panel] >= 0, f"Non-Negativity_{region}_{panel}")
        

# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    for region in regions:
        print(f"y[{region}] = {y[region].X}")
        for panel in types_of_panels:
            print(f"x[{region},{panel}] = {x[region,panel].X}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 45 rows, 20 columns and 95 nonzeros
Model fingerprint: 0x52d190b6
Variable types: 0 continuous, 20 integer (5 binary)
Coefficient statistics:
  Matrix range     [7e-02, 1e+100]
  Objective range  [3e+02, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+03, 4e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 43 rows and 17 columns
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)
Found heuristic solution: objective 4.764331e+07
Found heuristic solution: objective 4.671460e+07

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count

In [313]:
# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for i in regions:
        print(f'{i}, Build: {y[i].x}')
        for j in types_of_panels:
            print(f"{j}: {x[i, j].x} kwp System")
    print(f"Total Cost: ${model.objVal}")
else:
    print("No optimal solution found.")

Optimal solution found:
Central Region, Build: 1.0
Monocrystalline: 84653.0 kwp System
Polycrystalline: 5.0 kwp System
Thin-film: 0.0 kwp System
East Region, Build: 1.0
Monocrystalline: 25894.0 kwp System
Polycrystalline: -0.0 kwp System
Thin-film: -0.0 kwp System
North East Region, Build: 1.0
Monocrystalline: 22858.0 kwp System
Polycrystalline: -0.0 kwp System
Thin-film: -0.0 kwp System
North Region, Build: 1.0
Monocrystalline: 22359.0 kwp System
Polycrystalline: 5.0 kwp System
Thin-film: -0.0 kwp System
West Region, Build: 1.0
Monocrystalline: 19019.0 kwp System
Polycrystalline: 2.0 kwp System
Thin-film: 0.0 kwp System
Total Cost: $46714604.825


In [314]:
# Check if the model is infeasible
if model.status == GRB.INFEASIBLE:
    # Compute the IIS
    model.computeIIS()
    
    # Print the constraints in the IIS
    print("Constraints in the IIS:")
    for constr in model.getConstrs():
        if constr.IISConstr:
            print(constr.ConstrName)
    
    # Print the variable bounds in the IIS
    print("Variable bounds in the IIS:")
    for var in model.getVars():
        if var.IISLB:
            print(f"{var.VarName} (lower bound)")
        if var.IISUB:
            print(f"{var.VarName} (upper bound)")
else:
    print("Optimal solution found.")

Optimal solution found.
